In [2]:
import os
import re
import wave
import math
from pathlib import Path

def natural_sort_key(s):
    """自然排序函数，确保part10排在part9之后"""
    return [int(text) if text.isdigit() else text.lower() 
            for text in re.split(r'(\d+)', s)]

def get_file_size_mb(file_path):
    """获取文件大小（MB）"""
    if os.path.exists(file_path):
        return os.path.getsize(file_path) / (1024 * 1024)
    return 0

def estimate_batch_size(part_files, input_dir, max_batch_size_mb=3800):
    """估算每批次可包含的文件数，确保合并后不超过4GB（约3800MB）"""
    total_files = len(part_files)
    if total_files == 0:
        return 0
    
    # 计算所有文件的平均大小
    total_size = 0
    sample_count = min(10, total_files)  # 最多采样10个文件
    for i in range(sample_count):
        file_path = os.path.join(input_dir, part_files[i])
        total_size += get_file_size_mb(file_path)
    
    avg_file_size = total_size / sample_count if sample_count > 0 else 0
    if avg_file_size == 0:
        return 1  # 无法获取文件大小，保守估计每批次1个文件
    
    # 每批次最大文件数 = 最大批次大小(MB) / 单个文件大小(MB)
    files_per_batch = max(1, int(max_batch_size_mb / avg_file_size))
    print(f"估算每个文件平均大小: {avg_file_size:.2f} MB")
    print(f"每批次可容纳约 {files_per_batch} 个文件")
    
    return files_per_batch

def merge_batch(batch_files, batch_num, input_dir, output_dir):
    """合并一个批次的WAV文件"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    output_file = os.path.join(output_dir, f"蛊真人_{batch_num}.wav")
    
    # 打开第一个文件获取参数
    first_file = wave.open(os.path.join(input_dir, batch_files[0]), 'rb')
    params = first_file.getparams()
    
    # 创建输出文件
    output = wave.open(output_file, 'wb')
    output.setparams(params)
    
    # 合并批次内的所有文件
    total_frames = 0
    for i, part_file in enumerate(batch_files):
        file_path = os.path.join(input_dir, part_file)
        try:
            with wave.open(file_path, 'rb') as wf:
                # 验证参数一致性
                current_params = wf.getparams()
                if current_params != params:
                    print(f"警告: 文件 {part_file} 参数与第一个文件不一致")
                    print(f"  第一个文件: {params}")
                    print(f"  当前文件: {current_params}")
                    print("  将尝试继续合并，但可能导致音质问题")
                
                frames = wf.readframes(wf.getnframes())
                output.writeframes(frames)
                total_frames += wf.getnframes()
                
                # 显示进度
                if (i + 1) % 10 == 0 or i + 1 == len(batch_files):
                    print(f"  批次 {batch_num}: 已处理 {i+1}/{len(batch_files)} 个文件")
        except Exception as e:
            print(f"处理文件 {part_file} 时出错: {e}")
    
    output.close()
    first_file.close()
    
    # 计算并显示批次时长和大小
    sample_rate = params.framerate
    duration_seconds = total_frames / sample_rate
    hours = int(duration_seconds // 3600)
    minutes = int((duration_seconds % 3600) // 60)
    seconds = int(duration_seconds % 60)
    
    output_size = get_file_size_mb(output_file)
    
    print(f"批次 {batch_num} 合并完成!")
    print(f"  输出文件: {output_file}")
    print(f"  批次时长: {hours:02d}:{minutes:02d}:{seconds:02d}")
    print(f"  文件大小: {output_size:.2f} MB")
    
    return output_file

def batch_merge_wav(input_dir, output_dir_base, max_batch_size_mb=3800):
    """分批次合并WAV文件"""
    # 获取所有.part文件并排序
    part_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.wav') and '.part' in f],
                        key=natural_sort_key)
    
    if not part_files:
        print("错误: 未找到WAV分片文件")
        return []
    
    print(f"找到 {len(part_files)} 个音频片段")
    
    # 估算每批次文件数
    files_per_batch = estimate_batch_size(part_files, input_dir, max_batch_size_mb)
    total_batches = math.ceil(len(part_files) / files_per_batch)
    
    print(f"将分为 {total_batches} 个批次合并")
    
    # 创建批次输出目录
    output_dir = os.path.join(input_dir, output_dir_base)
    
    # 分批次合并
    batch_outputs = []
    for batch_num in range(total_batches):
        start_idx = batch_num * files_per_batch
        end_idx = min(start_idx + files_per_batch, len(part_files))
        batch_files = part_files[start_idx:end_idx]
        
        print(f"\n开始合并批次 {batch_num+1}/{total_batches}...")
        batch_output = merge_batch(batch_files, batch_num+1, input_dir, output_dir)
        batch_outputs.append(batch_output)
    
    # 计算总批次信息
    total_files = sum(len(batch) for batch in [part_files[i:i+files_per_batch] for i in range(0, len(part_files), files_per_batch)])
    print(f"\n合并完成! 共处理 {total_files} 个文件，生成 {total_batches} 个WAV文件")
    print(f"所有批次文件保存在: {output_dir}")
    
    return batch_outputs

if __name__ == "__main__":
    # 设置输入目录
    input_dir = "/app/sda1/xiangyue/work/IndexTTS-1.5-xy/蛊真人 (蛊真人) (Z-Library)"
    
    # 检查目录是否存在
    if not os.path.exists(input_dir):
        print(f"错误: 目录不存在 '{input_dir}'")
        exit(1)
    
    # 显示输入目录中的文件数量
    all_files = os.listdir(input_dir)
    print(f"输入目录包含 {len(all_files)} 个文件")
    
    # 执行分批次合并
    print("\n开始分批次合并WAV文件...")
    batch_outputs = batch_merge_wav(input_dir, "合并后的批次")
    
    # 显示合并后的批次文件列表
    if batch_outputs:
        print("\n合并后的批次文件列表:")
        total_size = 0
        for i, file in enumerate(batch_outputs):
            size_mb = get_file_size_mb(file)
            total_size += size_mb
            print(f"{i+1}. {os.path.basename(file)} - {size_mb:.2f} MB")
        
        print(f"\n总合并大小: {total_size:.2f} MB")


输入目录包含 86 个文件

开始分批次合并WAV文件...
找到 86 个音频片段
估算每个文件平均大小: 101.99 MB
每批次可容纳约 37 个文件
将分为 3 个批次合并

开始合并批次 1/3...
警告: 文件 蛊真人 (蛊真人) (Z-Library).wav.part1.wav 参数与第一个文件不一致
  第一个文件: _wave_params(nchannels=1, sampwidth=2, framerate=24000, nframes=53623808, comptype='NONE', compname='not compressed')
  当前文件: _wave_params(nchannels=1, sampwidth=2, framerate=24000, nframes=53897216, comptype='NONE', compname='not compressed')
  将尝试继续合并，但可能导致音质问题
警告: 文件 蛊真人 (蛊真人) (Z-Library).wav.part2.wav 参数与第一个文件不一致
  第一个文件: _wave_params(nchannels=1, sampwidth=2, framerate=24000, nframes=53623808, comptype='NONE', compname='not compressed')
  当前文件: _wave_params(nchannels=1, sampwidth=2, framerate=24000, nframes=54210560, comptype='NONE', compname='not compressed')
  将尝试继续合并，但可能导致音质问题
警告: 文件 蛊真人 (蛊真人) (Z-Library).wav.part3.wav 参数与第一个文件不一致
  第一个文件: _wave_params(nchannels=1, sampwidth=2, framerate=24000, nframes=53623808, comptype='NONE', compname='not compressed')
  当前文件: _wave_params(nchannels=1, sampwidth=2, framerat